# LangGraph Example: LLM with Tools
This Notebook uses an efficient tool calling approach that allows for parallel tool calling

### Notebook setup

In [1]:
# %pip install --upgrade langchain langsmith langgraph langchain_openai

In [2]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get OpenAI keys from .env file
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_ORGANIZATION"] = os.getenv("OPENAI_ORGANIZATION")

# Initialize LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Demos"

# Initialize OpenWeatherMap
OPENWEATHERMAP_API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")

### Set up the Tools


In [3]:
# Import things that are needed generically for tools
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

In [4]:
import requests


class City(BaseModel):
    city: str = Field(description="City")
    country: str = Field(description="Country code")


def get_current_weather(city: str, country: str) -> int:
    response = requests.get(
        f"http://api.openweathermap.org/data/2.5/weather?q={city},{country}&appid={OPENWEATHERMAP_API_KEY}"
    )
    data = response.json()
    temp_kelvin = data["main"]["temp"]
    temp_fahrenheit = (temp_kelvin - 273.15) * 9 / 5 + 32
    return int(temp_fahrenheit)


weather = StructuredTool.from_function(
    func=get_current_weather,
    name="Get_Weather",
    description="Get the current temperature from a city, in Fahrenheit",
    args_schema=City,
    return_direct=False,
)

In [5]:
class DifferenceInput(BaseModel):
    minuend: int = Field(
        description="The number from which another number is to be subtracted"
    )
    subtrahend: int = Field(description="The number to be subtracted")


def get_difference(minuend: int, subtrahend: int) -> int:
    return minuend - subtrahend


difference = StructuredTool.from_function(
    func=get_difference,
    name="Difference",
    description="Get the difference between two numbers",
    args_schema=DifferenceInput,
    return_direct=False,
)

In [6]:
tools = [weather, difference]

In [7]:
# Set up the tools to execute them from the graph
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

### Set up the agent


In [8]:
# Define the response schema for our agent
from langchain_core.pydantic_v1 import BaseModel, Field


class Response(BaseModel):
    """Final answer to the user"""

    warmest_city: str = Field(description="The warmest city and its current temperature")
    explanation: str = Field(
        description="How much warmer it is in the warmest city than the other cities"
    )

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)

# Create the OpenAI LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, streaming=True)

# Create the tools to bind to the model
tools = [convert_to_openai_function(t) for t in tools]
tools.append(convert_to_openai_function(Response))

# MODFIICATION: we're using bind_tools instead of bind_function
model = {"messages": RunnablePassthrough()} | prompt | llm.bind_tools(tools)

### Set up the Agent State

Everything from here onwards is new


In [10]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Sequence[BaseMessage]

### Set up the node actions


In [11]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import ToolMessage


# Define the function that determines whether to continue or not
def should_continue(state):
    last_message = state["messages"][-1]
    # If there are no tool calls, then we finish
    if "tool_calls" not in last_message.additional_kwargs:
        return "end"
    # If there is a Response tool call, then we finish
    elif any(
        tool_call["function"]["name"] == "Response"
        for tool_call in last_message.additional_kwargs["tool_calls"]
    ):
        return "end"
    # Otherwise, we continue
    else:
        return "continue"


# Define the function that calls the model
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": messages + [response]}


# Define the function to execute tools
def call_tool(state):
    messages = state["messages"]
    # We know the last message involves at least one tool call
    last_message = messages[-1]

    # We loop through all tool calls and append the message to our message log
    for tool_call in last_message.additional_kwargs["tool_calls"]:
        action = ToolInvocation(
            tool=tool_call["function"]["name"],
            tool_input=json.loads(tool_call["function"]["arguments"]),
            id=tool_call["id"],
        )

        # We call the tool_executor and get back a response
        response = tool_executor.invoke(action)
        # We use the response to create a FunctionMessage
        function_message = ToolMessage(
            content=str(response), name=action.tool, tool_call_id=tool_call["id"]
        )

        # Add the function message to the list
        messages.append(function_message)

    # We return a list, because this will get added to the existing list

    return {"messages": messages}

### Define the Graph


In [12]:
from langgraph.graph import StateGraph, END

# Initialize a new graph
graph = StateGraph(AgentState)

# Define the two Nodes we will cycle between
graph.add_node("agent", call_model)
graph.add_node("action", call_tool)

# Set the Starting Edge
graph.set_entry_point("agent")

# Set our Contitional Edges
graph.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)

# Set the Normal Edges
graph.add_edge("action", "agent")

# Compile the workflow
app = graph.compile()

### Run our graph


In [14]:
from langchain_core.messages import HumanMessage

inputs = {
    "messages": [
        HumanMessage(
            content="Where is it warmest: Austin, Texas; Tokyo; or Seattle? And by how much is it warmer than the other cities?"
        )
    ]
}
for output in app.with_config({"run_name": "LLM with Tools"}).stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [HumanMessage(content='Where is it warmest: Austin, Texas; Tokyo; or Seattle? And by how much is it warmer than the other cities?'), AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_giGADiZ1xHyI62bTTyb6PkRW', 'function': {'arguments': '{"city": "Austin", "country": "US"}', 'name': 'Get_Weather'}, 'type': 'function'}, {'index': 1, 'id': 'call_r5fYvUdmQotQgoHWY70WyBd8', 'function': {'arguments': '{"city": "Tokyo", "country": "JP"}', 'name': 'Get_Weather'}, 'type': 'function'}, {'index': 2, 'id': 'call_X5kkrMlQ8kShWUkffViQn2LF', 'function': {'arguments': '{"city": "Seattle", "country": "US"}', 'name': 'Get_Weather'}, 'type': 'function'}]})]}

---

Output from node 'action':
---
{'messages': [HumanMessage(content='Where is it warmest: Austin, Texas; Tokyo; or Seattle? And by how much is it warmer than the other cities?'), AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_giGADiZ1xHyI62